In [ ]:
key=""

# 结构化响应


In [ ]:
from typing import Union
import os
from pydantic import BaseModel

from pydantic_ai import Agent

os.environ["GEMINI_API_KEY"] = key
class Box(BaseModel):
    width: int
    height: int
    depth: int
    units: str

agent = Agent(
    "gemini-1.5-pro",
     system_prompt=(
        "Extract me the dimensions of a box, "
        "if you can't extract all data, ask the user to try again."
    ),
    output_type=Union[Box, str],  # type: ignore   
)

result=await agent.run('The box is 10x20x30')
result.output

Box(width=20, height=10, depth=30, units='undefined')

In [6]:
type(result.output)

__main__.Box

In [5]:
result =await agent.run('The box is 10x20x30 cm')
result.output

Box(width=30, height=20, depth=10, units='cm')

# 输出验证器


## 要求小写 SQL 的验证器


In [ ]:
from typing import Union
from pydantic import BaseModel
from pydantic_ai import Agent, RunContext, ModelRetry


class Success(BaseModel):
    sql_query: str


class InvalidRequest(BaseModel):
    error_message: str

Output = Union[Success, InvalidRequest]
agent = Agent[Output](
    "gemini-1.5-pro",
    output_type=Output,  # type: ignore
    system_prompt='Generate PostgreSQL flavored SQL queries based on user input.',
)

@agent.output_validator
async def validate_sql(ctx: RunContext, output: Output) -> Output:
    if isinstance(output, InvalidRequest):
        return output
    try:
        print(f'Executing query: {output.sql_query}')
        if output.sql_query.startswith('SELECT'):
            # 要求小写SQL
            raise ModelRetry('use lower case')
    except Exception as e:
        raise ModelRetry(f'Invalid query: {e}') from e
    else:
        return output


result = await agent.run(
    'get me users who were last active yesterday.'
)
print(result.output)


Executing query: SELECT * FROM users WHERE last_active = CURRENT_DATE - INTERVAL '1 day';
Executing query: select * from users where last_active = current_date - interval '1 day';
sql_query="select * from users where last_active = current_date - interval '1 day';"
